
# load dataframe

In [19]:
import pandas as pd

In [20]:
data= pd.read_csv('../EncodedData.csv')

In [21]:
alltags = [ 'clear', 'cloudy', 'haze', 'partly cloudy',        #the cloud coverage goes first!     
           'agriculture', 'artisinal mine', 'bare ground', 'blooming', 
           'blow down', 'cultivation', 'habitation', 'primary', 'road', 
        'selective logging', 'conventional mine', 'slashu burn','water']

In [22]:
data.head(1)

,image_name,tags,taglist,clear,cloudy,haze,partly cloudy,agriculture,artisinal mine,bare ground,...,blow down,cultivation,habitation,primary,road,selective logging,conventional mine,slashu burn,water,result_id
0,train_0,haze primary,"['haze', 'primary']",0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,'00100000000100000'


## tweak the names of the files so it's compatible

In [23]:
data['image_name'] = data['image_name'] + '.jpg'

In [24]:
data['image_name'].head(1)

0    train_0.jpg
Name: image_name, dtype: object

# load and preprocess data

## preprocessing

In [41]:
!ls -la

total 168
drwxr-xr-x   8 guillaumevanderwinden  staff    256 Nov 26 09:56 .
drwxr-xr-x  20 guillaumevanderwinden  staff    640 Nov 26 09:48 ..
-rw-r--r--@  1 guillaumevanderwinden  staff   6148 Nov 26 09:55 .DS_Store
drwxr-xr-x   4 guillaumevanderwinden  staff    128 Nov 26 09:44 .ipynb_checkpoints
-rw-r--r--   1 guillaumevanderwinden  staff      0 Nov 24 12:27 .keep
-rw-r--r--@  1 guillaumevanderwinden  staff  28518 Nov 26 09:56 VGGmodel.ipynb
-rw-r--r--   1 guillaumevanderwinden  staff  46883 Nov 25 16:51 gui-baseline_modeling.ipynb
drwxr-xr-x   4 guillaumevanderwinden  staff    128 Nov 26 09:55 sample


[the documents for imagedatagenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#random_transform)

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#instantiate the data generator and input the transformatons you want

datagen = ImageDataGenerator(rescale = 1./255,  #normalized
                             rotation_range = 45,
                             zoom_range= 0.3,   #so it will zoom to  0.7 and 1.3 times
                             brightness_range = [0.7, 1.3]
                            )


test_datagen=ImageDataGenerator(rescale = 1./255,  #normalized
                             rotation_range = 45,
                             zoom_range= 0.3,   #so it will zoom to  0.7 and 1.3 times
                             brightness_range = [0.7, 1.3],
                            )

## loading the images! 

I took 1200 +1 images for this one

In [50]:
#the X
train_generator = datagen.flow_from_dataframe(
                                            dataframe=data[0:700],
                                            directory="../notebooks/sample/train_1200",
                                            x_col='image_name',
                                            y_col=alltags,
                                            batch_size=32,
                                            seed=42,
                                            shuffle=False,
                                            class_mode="raw",  #look into this later
                                            target_size=(128,128)
)

Found 0 validated image filenames.


/Users/Gui/.pyenv/versions/3.7.7/envs/green-eye/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 700 invalid image filename(s) in x_col="image_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [27]:
# the validation set
valid_generator=test_datagen.flow_from_dataframe(
                                                dataframe=data[700:1000],
                                                directory="../notebooks/sample/",
                                                x_col="image_name",
                                                y_col=alltags,
                                                batch_size=32,
                                                seed=42,
                                                shuffle=False,
                                                class_mode="raw",
                                                target_size=(128,128)
)

Found 0 validated image filenames.


/Users/Gui/.pyenv/versions/3.7.7/envs/green-eye/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 300 invalid image filename(s) in x_col="image_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [28]:
#the test set
test_generator=test_datagen.flow_from_dataframe(
                                                dataframe=data[1000:1200],
                                                directory="../notebooks/sample/",
                                                x_col="image_name",
                                                y_col=alltags,
                                                batch_size=32,
                                                seed=42,
                                                shuffle=False,
                                                class_mode="raw",
                                                target_size=(128,128)
)

Found 0 validated image filenames.


/Users/Gui/.pyenv/versions/3.7.7/envs/green-eye/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 200 invalid image filename(s) in x_col="image_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [34]:
#check an image for prediction!
image_number = 1200   #the 1200th image

sample_predict=test_datagen.flow_from_dataframe(
                                                dataframe=data[image_number:image_number+1],
                                                directory="../greeneye/",
                                                x_col="image_name",
#                                                y_col=alltags,
                                                batch_size=32,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=(128,128)
)

#store the image for later check
from tensorflow.keras.preprocessing.image import load_img

sample= data['image_name'][image_number]
sample_image = load_img(f'../train_1200/{sample}')

Found 0 validated image filenames.


FileNotFoundError: [Errno 2] No such file or directory: '../train_1200/train_1200.jpg'

# make the model

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [17]:
# VGG16

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


#def VGG_16(weights_path=None):

model = Sequential()
model.add(Conv2D(input_shape=(128,128,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=17, activation="softmax"))

In [75]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [76]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_56 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 32, 32, 256)      

# run the model

In [1]:
history = model.fit(train_generator,
          batch_size= 32,
          validation_data=valid_generator,
          epochs = 10
         )

NameError: name 'model' is not defined

# evaluate the model

In [ ]:
model.evaluate(test_generator)

In [ ]:
import matplotlib.pyplot as plt
def plot_loss_accuracy(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()

    plot_loss_accuracy(history)

# predict the model

In [ ]:
a= model.predict(sample_predict)

In [ ]:
a

In [ ]:
#check which image it was
sample_image

In [ ]:
#it's properties are:
data.iloc[image_number,3:-1]

# save the model

In [ ]:
from tensorflow.keras import models

models.save_model(model, 
                  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',  #filepath
                  overwrite=True, 
                  include_optimizer=True, #important hehe
)     